In [1]:
from PyPDF2 import PdfWriter, PdfReader
import io
import os
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.colors import white  # Import for setting the font color to white

In [2]:
# Adjust these paths and settings as necessary
certemplate = "/Users/template.pdf" # PDF template location
excelfile = "/Users/test _data.xlsx" # Excel file data
font_path = "/Users/Roboto-Regular.ttf" # Preferred font 
output_folder = "certificates" # where should the certificate PDFs be saved

# Text placement and appearance settings
horz = 300  # Horizontal position for the text; adjust as necessary
vert = 220  # Vertical position for the text; adjust as necessary
font_size = 20  # Font size; adjust as necessary
hash_font_size = 6  # Font size for the hash, ensure this line is included

# Ensure the output directory exists
os.makedirs(output_folder, exist_ok=True)

# Register the font
pdfmetrics.registerFont(TTFont('CustomFont', font_path))

# Load the Excel file
data = pd.read_excel(excelfile)

# Right-end alignment for the hash
right_margin = 84  # Distance from the right edge where the hash ends. Adjust as needed.

# Calculate the page width for landscape orientation
page_width = landscape(letter)[0]

# Iterate over the rows of the DataFrame
for index, row in data.iterrows():
    full_name = f"{row['First Name'].strip()} {row['Last Name'].strip()}"
    certificate_hash = row['hash'].strip()  # Assuming 'Hash' is the column name for the hash values

    # Create a PDF with the participant's name and hash
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=landscape(letter))
    can.setFont('CustomFont', font_size)
    
    # Draw the participant's name
    # Adjust `horz` and `vert` as needed
    can.drawString(horz, vert, full_name)
    
    # Set the font color to white for the hash
    can.setFillColor(white)
    can.setFont('CustomFont', hash_font_size)

    # Calculate the width of the hash text
    hash_text = certificate_hash
    hash_text_width = pdfmetrics.stringWidth(hash_text, 'CustomFont', hash_font_size)
    
    # Adjust hash_horz to right-align the hash text
    hash_horz = page_width - right_margin - hash_text_width  # This positions the end of the hash at a fixed distance from the right edge

    
    # Calculate the top right position for the hash
    # These values might need adjustment
    #hash_horz = 505  # Horizontal position for the hash, adjust as necessary
    hash_vert = 380  # Vertical position for the hash, adjust as necessary
    can.drawString(hash_horz, hash_vert, certificate_hash)

    can.save()

    # Move to the beginning of the StringIO buffer
    packet.seek(0)
    new_pdf = PdfReader(packet)

    # Read the template PDF
    existing_pdf = PdfReader(certemplate)
    output = PdfWriter()

    # Merge the new PDF with the template
    page = existing_pdf.pages[0]
    page.merge_page(new_pdf.pages[0])
    output.add_page(page)

    # Save the modified PDF
    destination = os.path.join(output_folder, f"Certificate - {full_name}.pdf")
    with open(destination, "wb") as outputStream:
        output.write(outputStream)

    print(f"Created certificate: {destination}")

TTFError: Can't open file "/Users/Roboto-Regular.ttf"